## knock 042 過去n件のデータを結合する

In [4]:
import polars as pl
pl.Config.set_tbl_cols(-1)# 列が省略されないようにする

polars.config.Config

### データを読み込む

In [5]:
df_receipt = pl.read_csv("../docker/work/data/receipt.csv")
display(df_receipt.head())

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
i64,i64,str,i64,i64,str,str,i64,i64
20181103,1541203200,"""S14006""",112,1,"""CS006214000001…","""P070305012""",1,158
20181118,1542499200,"""S13008""",1132,2,"""CS008415000097…","""P070701017""",1,81
20170712,1499817600,"""S14028""",1102,1,"""CS028414000014…","""P060101005""",1,170
20190205,1549324800,"""S14042""",1132,1,"""ZZ000000000000…","""P050301001""",1,25
20180821,1534809600,"""S14025""",1102,2,"""CS025415000050…","""P060102007""",1,90


### ノック

In [7]:
# 月毎の合計売上
df_group_by = (
    df_receipt[["sales_ymd", "amount"]]
    # 月毎の合計売上
    .group_by(by = "sales_ymd")
    .agg( pl.sum("amount") )
    .sort(by = "sales_ymd")
)

(
    pl.concat(items = [df_group_by,
                       df_group_by.shift(n = 1).rename(mapping = {"sales_ymd": "lag_ymd_1", "amount": "lag_amount_1"}),
                       df_group_by.shift(n = 2).rename(mapping = {"sales_ymd": "lag_ymd_2", "amount": "lag_amount_2"}),
                       df_group_by.shift(n = 3).rename(mapping = {"sales_ymd": "lag_ymd_3", "amount": "lag_amount_3"})],
            how = "horizontal")
    # 4営業日分のデータが無い行を削除する
    .drop_nulls()
    .head(n = 10)
)

sales_ymd,amount,lag_ymd_1,lag_amount_1,lag_ymd_2,lag_amount_2,lag_ymd_3,lag_amount_3
i64,i64,i64,i64,i64,i64,i64,i64
20170104,36165,20170103,27503,20170102,24165,20170101,33723
20170105,37830,20170104,36165,20170103,27503,20170102,24165
20170106,32387,20170105,37830,20170104,36165,20170103,27503
20170107,23415,20170106,32387,20170105,37830,20170104,36165
20170108,24737,20170107,23415,20170106,32387,20170105,37830
20170109,26718,20170108,24737,20170107,23415,20170106,32387
20170110,20143,20170109,26718,20170108,24737,20170107,23415
20170111,24287,20170110,20143,20170109,26718,20170108,24737
20170112,23526,20170111,24287,20170110,20143,20170109,26718
